# Couple a simple single column model to the RRTMG scheme and explore ECS

The thermal structure of the atmosphere is assumed to follow the pseudoadiabatic lapse rate in the troposphere, with an overlying isothermal stratosphere at a fixed tropopause temperature, following Seeley and Jeevanjee (2020).

In [75]:
%matplotlib inline 
import climlab
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate as sp  #Gives access to the ODE integration package

In [119]:
class Dummy:
    pass

params = Dummy()

params.eps   = 0.622 # molar mass ratio of vapour to total moist air
params.L     = 2260000 # Latent heat of vap, water, J/kg

params.R_a   = 287.058 # gas constant for dry air, J/kg/K
params.R_c   = 461.5  # gas constant for water vapour, J/kg/K
params.cp_a  = 1004 # specific heat capacity of air, J/kg/K

In [135]:
# Define functions

def q(T, P): # specific humidity at saturation (using Clausius-Clapeyron)
    return climlab.utils.thermo.qsat(T, P)

def moist_adiabat(T, P): # Pressure coords gives dT/dP (if using the T/P prefactor)
    return np.divide(T, P) * np.divide(params.R_a, params.cp_a) * np.divide(1 + np.divide(params.L*q(T, P), params.R_a * T), 
                                                                            1 + np.divide(params.L**2 * q(T, P), params.cp_a * params.R_c * T**2))

# Or, equivalently...
def pseudoadiabat(T,p):
    return climlab.utils.thermo.pseudoadiabat(T, p)

def dry_adiabat(T):
    return np.divide(params.params.R_a, cp_a)

def generate_idealized_temp_profile(SST, Tstrat, plevs):
    solution = sp.odeint(pseudoadiabat, SST, np.flip(plevs))
    temp = solution.reshape(-1)
    temp[np.where(temp<Tstrat)] = Tstrat
    return temp

In [265]:
def test_olr(SST, CO2):
    ## RRTMG
    #  Couple water vapor to radiation
    ## climlab setup
    # create surface and atmosperic domains
    sfc, atm = climlab.domain.single_column(num_lev=30, num_lat=1, water_depth=1.)

    # create atmosheric state
    state = AttrDict()

    # assign surface temperature and vertical temperature profiles
    Ts_ = climlab.Field(np.array([SST]), domain=sfc)
    state['Ts'] = Ts_

    temp = generate_idealized_temp_profile(SST, 
                                           params.Tstrat, 
                                           atm.axes['lev'].points)
    Tatm = climlab.Field(temp, domain=atm)
    state['Tatm'] = Tatm

    #  Create a parent process
    rce = climlab.TimeDependentProcess(state=state)
    ## Create individual physical process models:
    #  fixed relative humidity
    h2o = climlab.radiation.ManabeWaterVapor(state=state)
    # RRTMG radiation
    rad = climlab.radiation.RRTMG(state=state, specific_humidity=h2o.q, albedo=0.3, ozone_file=None)
    rad.subprocess['LW'].absorber_vmr = {'CO2':CO2*1.e-6*np.ones(30),
                                         'CH4':0.,
                                         'N2O':0.,
                                         'O2':0.,
                                         'CFC11':0.,
                                         'CFC12':0.,
                                         'CFC22':0.,
                                         'CCL4':0.,
                                         'O3':0.
                                        }
    
    # Couple the models
    rce.add_subprocess('Radiation', rad)
    rce.add_subprocess('H2O', h2o)

    rce.compute()
    olr = rce.OLR[0]
    #print(olr,end="\n")
    return olr

In [282]:
OLR0 = test_olr2(SST=288,CO2=280)
OLR0

374.6935542136357

In [283]:
test_olr2(SST=300,CO2=130)

441.312881887575

In [246]:
%%time

TEMPS = np.linspace(280, 285, 6) #np.linspace(280, 320, 41)
CO2_arr = np.zeros(len(TEMPS))

co2_init = 280
for idx,sst in enumerate(TEMPS):
    print(sst)
    olr = test_olr(SST=sst,CO2=co2_init)
    co2=co2_init
    while abs(olr-OLR0)>0.1:
        if olr-OLR0>0:
            co2+=1
            olr = test_olr(SST=sst,CO2=co2)
        if olr-OLR0<0:
            co2-=1
            olr = test_olr(SST=sst,CO2=co2)
        print(co2)
    CO2_arr[idx] = co2

280.0
279
278
277
276
275
274
273
272
271
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256
255
254
253
252
251
250
249
248
247
246
245
244
243
242
241
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
224
223
222
221
220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8


KeyboardInterrupt: 

In [242]:
CO2_arr

array([0., 0., 0., 0., 0., 0.])

In [253]:
test_olr(280)

TypeError: test_olr() missing 1 required positional argument: 'CO2'